# Random Forest

## 비지니스 목적: 이탈 고객 식별 & 방지
### 핵심적으로 봐야 할 지표
1. 클래스 1 Recall (이탈 고객 재현율)
- 놓치는 이탈 고객을 최소화하기 위해 중요
- Recall이 높을수록 실제 이탈 고객을 더 많이 잡아냄
- 마케팅/혜택 대상 누락 방지

2. 클래스 1 Precision (이탈 고객 정밀도)
- ‘이탈 위험’으로 분류한 고객 중 실제 이탈하는 비율
- Precision이 높으면 불필요한 혜택 비용, 자원 낭비 방지

3. Balanced Accuracy
- 클래스 0(비이탈)과 클래스 1(이탈) 성능 균형
- 한 쪽 클래스만 잘 맞추는 모델 방지  

$Balanced\ Accuracy = \frac{Recall_0 + Recall_1}{2}$

4. ROC-AUC
- 고객을 위험도 순서로 정렬해 상위 위험군부터 조치 가능
- 마케팅 자원을 효율적으로 배분할 수 있음

5. PR-AUC (클래스 1 기준)
- Precision과 Recall의 관계를 종합적으로 반영
- 실제 캠페인 효율성 평가에 적합

---

## 범주형 인코딩 방법 추천표

| Feature | Logistic Regression | Random Forest | XGBoost | LightGBM | SVM | 비고 |
|---------|--------------------|--------------|---------|----------|-----|------|
| **Home Ownership (2)** | One-Hot **(권장)** | One-Hot **(권장)** | One-Hot **(권장)** | Native Categorical **(권장)** / One-Hot **(대체 가능)** | One-Hot **(권장)** | 이진 변수이지만 회귀 계수 해석 편의를 위해 One-Hot 선호 |
| **Ethnicity (73)** | One-Hot **(권장)** | Label Encoding **(대체 가능)** | Label Encoding **(권장)** | Native Categorical **(권장)** | One-Hot **(권장)** | 범주 수 많음 → LR/SVM은 One-Hot, 트리계열은 Label/Native |
| **Language (38)** | One-Hot **(권장)** | Label Encoding **(대체 가능)** | Label Encoding **(권장)** | Native Categorical **(권장)** | One-Hot **(권장)** | Ethnicity와 유사하게 처리 |
| **City (56)** | One-Hot **(권장)** | Label Encoding **(대체 가능)** | Label Encoding **(권장)** | Native Categorical **(권장)** | One-Hot **(권장)** | 도시 정보, 범주 많음 |
| **County (4)** | One-Hot **(권장)** | One-Hot **(권장)** | One-Hot **(권장)** | Native Categorical **(권장)** | One-Hot **(권장)** | 범주 수 작음 |
| **weekly fee (14)** | One-Hot **(권장)** | Label Encoding **(대체 가능)** | Label Encoding **(권장)** | Native Categorical **(권장)** | One-Hot **(권장)** | 금액 범위라 순서형일 수 있으나 코드상 문자열이므로 범주형 처리 |
| **Deliveryperiod (22)** | One-Hot **(권장)** | Label Encoding **(대체 가능)** | Label Encoding **(권장)** | Native Categorical **(권장)** | One-Hot **(권장)** | 배달 패턴 정보 |
| **Nielsen Prizm (9)** | One-Hot **(권장)** | Label Encoding **(대체 가능)** | Label Encoding **(권장)** | Native Categorical **(권장)** | One-Hot **(권장)** | 인구 통계 세그먼트 |
| **Source Channel (50)** | One-Hot **(권장)** | Label Encoding **(대체 가능)** | Label Encoding **(권장)** | Native Categorical **(권장)** | One-Hot **(권장)** | 마케팅 채널 정보 |

---

# 1. 그냥 학습 시키기

## 인코딩
<특이사항>
- 한 번에 여러 컬럼을 인코딩하게 해주는 파이프라인을 제공하는 ColumnTransformer 사용 
- LabelEncoder는 1D array 즉 특성 한 개만 인코딩 가능하기 때문에 2D array 인코딩이 가능한 OrdinalEncoder 사용

In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

newspaper_df = pd.read_csv('./data/newspaper_preprocessed.csv')

# dummy for Children 컬럼 drop
newspaper_df = newspaper_df.drop('dummy for Children', axis=1)
#display(newspaper_df.head())

# 데이터 분리 (특성-타겟)
y = newspaper_df['is_churned'].astype(int)
X = newspaper_df.drop(columns=['is_churned'])

# Random Forest 인코딩 규칙
onehot_cols = ['Home Ownership', 'County', 'weekly fee', 'Nielsen Prizm']
label_cols  = ['Ethnicity', 'Language', 'City', 'Deliveryperiod', 'Source Channel']

# 수치형 특성
numeric_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()

# Keep only existing columns
onehot_cols  = [c for c in onehot_cols  if c in X.columns]
label_cols   = [c for c in label_cols   if c in X.columns]
numeric_cols = [c for c in numeric_cols if c in X.columns]

# 인코더 & 스케일러 
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1) 

# ColumnTransformer
preprocess_rf = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_cols),  # 수치형 → 스케일링
        ('ohe', ohe, onehot_cols),      # One-Hot Encoding
        ('ord', ord_enc, label_cols)    # Ordinal Encoding == multi-column label encoding
    ],
    remainder='drop'
)

# 전체 데이터셋 인코딩
X_encoded= preprocess_rf.fit_transform(X)

## 확인을 위해 다시 DataFrame으로 변환
""" 
# 인코딩 된 컬럼들
num_feats  = numeric_cols
ohe_feats  = preprocess_rf.named_transformers_['ohe'].get_feature_names_out(onehot_cols).tolist()
ord_feats  = label_cols

encoded_cols = num_feats + ohe_feats + ord_feats
newspaper_df = pd.DataFrame(encoded_array, columns=encoded_cols, index=X.index)

# 타겟 변수 추가
newspaper_df['is_churned'] = y.values
"""
print(X_encoded.shape)

(15438, 38)


In [2]:
# Train-test split
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold

# Imbalance한 y값을 고려해 stratified split 진행 (클래스 비율 유지)
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, stratify=y, random_state=42
)

print("Train class ratio:", y_train.mean().round(4), "(is_churned==1 비율)")
print("Test  class ratio:", y_test.mean().round(4), "(is_churned==1 비율)")

Train class ratio: 0.8054 (is_churned==1 비율)
Test  class ratio: 0.8054 (is_churned==1 비율)


학습 및 평가

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
# 평가 지표
from sklearn.metrics import (
    balanced_accuracy_score,   # 불균형 데이터용: 클래스별 재현율 평균
    roc_auc_score,             # 순위(랭킹) 성능
    average_precision_score,   # PR-AUC 계산(평균정밀도)
    recall_score,              # 재현율
    confusion_matrix,          # 혼동 행렬
    classification_report,     # 정밀도/재현율/F1 종합 리포트
    make_scorer               # 커스텀 스코어러 생성
)


# 보조 지표: 소수 클래스(여기서는 클래스 0) PR-AUC 계산 함수
def pr_auc_minority_scorer(y_true, y_score_pos1, **kwargs):
    # y_score_pos1: 클래스 1(다수) 확률 → 클래스 0 지표를 위해 반전
    return average_precision_score(1 - y_true, 1 - y_score_pos1)

# 스코어링 딕셔너리
scoring = {
    'balanced_acc': make_scorer(balanced_accuracy_score),      # 모델 선택 기준
    'roc_auc': 'roc_auc',                                      # 전체 순위 성능 확인
    'recall_minority': make_scorer(recall_score, pos_label=0), # 클래스 0 재현율
    'pr_auc_minority': make_scorer(pr_auc_minority_scorer),    # 클래스 0 PR-AUC
}


# 모델 & 파이프라인 
rf = RandomForestClassifier(random_state=42, n_jobs=-1, bootstrap=True)
pipe = Pipeline([('clf', rf)])

# 5) 하이퍼파라미터 검색 범위 (bootstrap=True 고정, max_samples 활용)
#    - n_estimators: 충분한 안정성 vs 시간 균형
#    - max_depth / min_samples_*: 과적합 제어
#    - max_features: 전형적 선택지 + 비율 옵션
#    - class_weight: 불균형 대응 (다수=1, 소수=0)
param_grid = {
    'clf__n_estimators': [400, 800],
    'clf__max_depth': [None, 16, 24],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4, 8],
    'clf__max_features': ['sqrt', 'log2', 0.5],
    'clf__class_weight': ['balanced'],
    'clf__max_samples': [None, 0.7, 0.9],  # 부트스트랩 샘플 비율
}

# 교차검증 전략 (계층적 K-Fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
gcv = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=scoring,
    refit='balanced_acc',   # 최종 선택 기준
    cv=cv,
    n_jobs=-1, # cpu 코어수 극대화 (학습 속도 빠르게 하려고)
    verbose=1 # 학습 진행 상황 설명
)

# 학습 
gcv.fit(X_train, y_train)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('clf',
                                        RandomForestClassifier(n_jobs=-1,
                                                               random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__class_weight': ['balanced'],
                         'clf__max_depth': [None, 16, 24],
                         'clf__max_features': ['sqrt', 'log2', 0.5],
                         'clf__max_samples': [None, 0.7, 0.9],
                         'clf__min_samples_leaf': [1, 2, 4, 8],
                         'clf__min_samples_split': [2, 5, 10],
                         'clf__n_estimators': [400, 800]},
             refit='balanced_acc',
             scoring={'balanced_acc': make_scorer(balanced_accuracy_score, response_method='predict'),
                      'pr_auc_minority': make_scorer(pr_auc_minority_scorer, response_method='predict'),
                      'recall_minority': make_scorer(recall_score, response_method='predict', pos_label=0),
                      'roc_auc': 'roc_auc'},
             verbose=1)

In [4]:
print("최적 balanced_acc (CV):", round(gcv.best_score_, 4))
print("최적 하이퍼파라미터:", gcv.best_params_)

# 테스트 평가
best_model = gcv.best_estimator_
y_proba = best_model.predict_proba(X_test)[:, 1]
y_pred  = best_model.predict(X_test)

print("\n=== 테스트 세트 성능 ===")
print("Balanced Acc:", round(balanced_accuracy_score(y_test, y_pred), 4))
print("ROC-AUC     :", round(roc_auc_score(y_test, y_proba), 4))
print("클래스 0 Recall:", round(recall_score(y_test, y_pred, pos_label=0), 4))
print("클래스 0 PR-AUC :", round(pr_auc_minority_scorer(y_test, y_proba), 4))
print("\n혼동 행렬:\n", confusion_matrix(y_test, y_pred))
print("\n분류 보고서:\n", classification_report(y_test, y_pred, digits=4))

최적 balanced_acc (CV): 0.7672
최적 하이퍼파라미터: {'clf__class_weight': 'balanced', 'clf__max_depth': 24, 'clf__max_features': 'sqrt', 'clf__max_samples': None, 'clf__min_samples_leaf': 8, 'clf__min_samples_split': 2, 'clf__n_estimators': 400}

=== 테스트 세트 성능 ===
Balanced Acc: 0.7606
ROC-AUC     : 0.8551
클래스 0 Recall: 0.6889
클래스 0 PR-AUC : 0.6023

혼동 행렬:
 [[ 414  187]
 [ 417 2070]]

분류 보고서:
               precision    recall  f1-score   support

           0     0.4982    0.6889    0.5782       601
           1     0.9171    0.8323    0.8727      2487

    accuracy                         0.8044      3088
   macro avg     0.7077    0.7606    0.7254      3088
weighted avg     0.8356    0.8044    0.8154      3088



### 1. Balanced Accuracy
- **CV**: 0.7672 → 교차검증에서 평균 약 76.7%의 균형 정확도 달성  
- **Test**: 0.7606 → 테스트 데이터에서도 약 76.1% 유지  

- **의미**: 각 클래스별 Recall의 평균값  

- **이 데이터에서 중요한 이유**  
    - `is_churned`가 80:20 불균형 분포  
    - 단순 Accuracy는 다수 클래스(80%) 예측에 치우쳐도 높게 나올 수 있음  
    - Balanced Accuracy는 두 클래스의 성능을 동일 비중으로 평가  


### 2. ROC-AUC (0.8551)
- **의미**: 임계값과 상관없이, 모델이 양성 클래스(이탈 고객) 점수를 음성 클래스(비이탈)보다 높게 매기는 비율  
- 1.0이면 완벽 구분, 0.5면 무작위 예측 수준  
- **장점**: 분류 기준(threshold)을 정하기 전, **모델의 순위 매김(ranking) 능력**을 평가 가능  


### 3. Recall (클래스 0 = 비이탈 고객)
- **값**: 0.6889 → 실제 비이탈 고객 중 약 68.9%를 올바르게 예측  
- **해석**:  
    - 클래스 0이 소수 클래스이므로, **소수 클래스의 재현율** 확인은 모델 공정성 평가에 중요  
    - **Precision이 0.50 수준** → “비이탈”로 예측된 고객 중 절반은 실제 이탈 고객 → 잘못된 안심 효과 가능성 있음  
    - 비즈니스 목표가 “이탈 방지”라면, **클래스 1(이탈)** Recall과 함께 해석 필요  


### 4. Recall (클래스 1 = 이탈 고객)
- **값**: 0.8323 → 실제 이탈 고객의 약 83.2%를 올바르게 예측  
- **해석**:  
    - Precision이 0.92로 매우 높음 → "이탈"로 분류된 고객 대부분이 실제 이탈 고객  
    - 놓치는 이탈 고객(FN)이 16.8% 존재 → 이 비율을 낮추면 이탈 방지 효과 상승  


### 5. PR-AUC (클래스 0, 0.6023)
- **의미**: Precision-Recall 곡선 아래 면적  
- **이유**:  
    - **불균형 데이터**에서는 ROC-AUC보다 PR-AUC가 실제 성능을 더 현실적으로 반영  
    - 0.60 수준은 “보통” 정도이며, 소수 클래스 예측의 Precision·Recall 모두 개선 여지 있음  


### 6. 혼동 행렬 해석
- **TN (414)**: 비이탈을 정확히 예측  
- **FP (187)**: 비이탈인데 이탈로 잘못 예측 → 불필요한 유지 마케팅 발생 가능  
- **FN (417)**: 이탈인데 비이탈로 잘못 예측 → **이탈 방지 실패로 직결**  
- **TP (2070)**: 이탈을 정확히 예측 → 방지 캠페인 타겟 가능  


### <종합 해석>
- **이탈 고객(클래스 1)**: Precision 92%, Recall 83%로 매우 우수  
- **비이탈 고객(클래스 0)**: Precision이 낮아(50%) 오분류 리스크 존재  
    - = 비이탈 고객이 정말 잔류를 할지에 대한 예측력이 떨어짐  
    - **이를 보완하기 위해서 oversampling(SMOTENC 등)을 진행할 당위성이 충분히 존재**

---

# 2. 학습 시 Oversampling 적용
<SMOTENC (=범주형 데이터를 포함한 SMOTE) 기법을 이용>

-현재 데이터에서 is_churned=0의 비율이 매우 적어 이를 보완하기 위해 SMOTENC와 같은 오버샘플링 기법을 적용하려고 함. 

- 이때 중요한 점은 교차검증(CV) 환경에서는 반드시 SMOTENC를 파이프라인(Pipeline)에 포함하는 것이 좋다는 것. 

- 그 이유는 첫째, 데이터 누수를 방지하기 위함. 전체 데이터에서 한 번 오버샘플링을 하고 이를 폴드별로 나누면, 각 폴드가 동일한 합성 데이터를 공유하게 되어 검증 점수가 실제보다 높게 나오는 문제가 발생. 
    - 파이프라인에 포함하면 각 폴드의 훈련 데이터에만 SMOTENC를 적용하여 매번 새로운 합성 데이터를 만들기 때문에 이러한 누수를 방지할 수 있음. 

    - 둘째, 오버샘플링 방식과 모델 하이퍼파라미터를 동시에 최적화할 수 있습니다. SMOTENC 자체도 k_neighbors나 sampling_strategy 등 성능에 영향을 주는 하이퍼파라미터를 가지므로, 파이프라인으로 묶어 그리드서치 시 함께 탐색하는 것이 효과적. 

    - 마지막으로, 코드의 재현성과 간결성을 높히기 가능. 파이프라인을 사용하면 모델 학습 과정이 명확해져 팀원들이 동일한 절차를 손쉽게 재현할 수 있습니다.

- 단, 교차검증을 사용하지 않고 단일 train/valid 분할만으로 실험하는 경우에는, 훈련 세트에서만 오버샘플링을 한 뒤 모델을 학습하고 원래 테스트 세트로 평가하는 절차를 따르더라도 무방하지만 CV와 하이퍼파라미터 튜닝을 함께 진행한다면 파이프라인 사용이 사실상 필수라고 보아야 함.

In [14]:
#!pip install imbalanced-learn

In [8]:
import os
os.environ["SCIPY_ARRAY_API"] = "1"
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
X_raw = X

# SMOTENC & ColumnTransformer에서 쓸 "컬럼 인덱스"
onehot_idx  = [X_raw.columns.get_loc(c) for c in onehot_cols]
label_idx   = [X_raw.columns.get_loc(c) for c in label_cols]
numeric_idx = [X_raw.columns.get_loc(c) for c in numeric_cols]

# SMOTENC에 전달할 '범주형' 컬럼 인덱스 (OHE 대상 + Ordinal 대상)
#categorical_idx_for_smote = np.array(onehot_idx + label_idx, dtype=int)
categorical_idx_for_smote = list(onehot_idx + label_idx)

# SMOTENC 정의
smote = SMOTENC(
    categorical_features=categorical_idx_for_smote,
    sampling_strategy='auto',    # 소수 클래스를 다수에 가깝게
    k_neighbors=5,
    random_state=42
)

# train/test 분리
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.2, stratify=y, random_state=42
)


preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_idx),  # 수치형 → 스케일링
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False), onehot_idx),
        ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), label_idx),
    ],
    remainder='drop'
)


rf = RandomForestClassifier(random_state=42, n_jobs=1, bootstrap=True)


# 6) SMOTENC(원본) → 전처리(인코딩) → RF
pipe = ImbPipeline(steps=[
    ('smote', smote),
    ('prep', preprocess),
    ('clf', rf),
])


# 스코어러 설정 (소수 클래스=0 기준 PR-AUC 포함)
def pr_auc_minority(y_true, y_score_pos1, **kwargs):
    # average_precision_score는 "양성=1" 기준 → 0 기준으로 보려면 반전
    return average_precision_score(1 - y_true, 1 - y_score_pos1)

scoring = {
    'balanced_acc': make_scorer(balanced_accuracy_score),
    'roc_auc': 'roc_auc',
    'recall_minority': make_scorer(recall_score, pos_label=0),
    'pr_auc_minority': make_scorer(pr_auc_minority, needs_proba=True),
}

# 하이퍼파라미터 그리드 (이전 최적값 주변 + SMOTENC 동시 탐색)
param_grid = {
    # RF: 이전 최적 근처를 촘촘히 재탐색
    'clf__n_estimators': [600, 800],
    'clf__max_depth': [16, 24],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [4, 8, 12],        # 합성 노이즈 완충을 위해 약간 키움
    'clf__max_features': ['sqrt'],
    'clf__class_weight': [None],    # 오버샘플링과의 조합 비교
    'clf__max_samples': [None, 0.9],

    # SMOTENC: 데이터 밀도에 민감 → 함께 튜닝 권장
    'smote__k_neighbors': [3, 5],
    'smote__sampling_strategy': ['auto', 0.5],  # 소수 클래스 비율(0.5=40%)
}

# 교차검증 & 그리드서치
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

gcv = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=scoring,
    refit='balanced_acc',     # 최종 선택 기준(두 클래스 균형 성능)
    cv=cv,
    n_jobs=-1,
    verbose=1,
    pre_dispatch='2*n_jobs',  # RAM 초과 억제
    error_score='raise'
)

# 학습
gcv.fit(X_train_raw, y_train)

print("최적 balanced_acc (CV):", round(gcv.best_score_, 4))
print("최적 하이퍼파라미터:", gcv.best_params_)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


/opt/anaconda3/envs/ml_env/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


최적 balanced_acc (CV): 0.7549
최적 하이퍼파라미터: {'clf__class_weight': None, 'clf__max_depth': 16, 'clf__max_features': 'sqrt', 'clf__max_samples': None, 'clf__min_samples_leaf': 12, 'clf__min_samples_split': 2, 'clf__n_estimators': 600, 'smote__k_neighbors': 3, 'smote__sampling_strategy': 'auto'}


In [10]:
# 평가 (원본 테스트셋: 여기에서는 클래스 불균형이 그대로 남아있음)
best_model = gcv.best_estimator_
y_proba = best_model.predict_proba(X_test_raw)[:, 1]
y_pred  = best_model.predict(X_test_raw)

print("\n=== 테스트 성능 (SMOTENC 포함) ===")
print("Balanced Acc:", round(balanced_accuracy_score(y_test, y_pred), 4))
print("ROC-AUC     :", round(roc_auc_score(y_test, y_proba), 4))
print("클래스 0 Recall:", round(recall_score(y_test, y_pred, pos_label=0), 4))
print("클래스 0 PR-AUC :", round(pr_auc_minority(y_test, y_proba), 4))
print("\n혼동 행렬:\n", confusion_matrix(y_test, y_pred))
print("\n분류 보고서:\n", classification_report(y_test, y_pred, digits=4))


=== 테스트 성능 (SMOTENC 포함) ===
Balanced Acc: 0.7606
ROC-AUC     : 0.8393
클래스 0 Recall: 0.7121
클래스 0 PR-AUC : 0.574

혼동 행렬:
 [[ 428  173]
 [ 475 2012]]

분류 보고서:
               precision    recall  f1-score   support

           0     0.4740    0.7121    0.5691       601
           1     0.9208    0.8090    0.8613      2487

    accuracy                         0.7902      3088
   macro avg     0.6974    0.7606    0.7152      3088
weighted avg     0.8339    0.7902    0.8044      3088



메모리 누수가 발생했으므로 정확한 학습 및 평가가 이루어졌다고 보기 힘듬

---

# 3. RAM 누수 억제 학습
- 하이퍼파라미터 조정
- k-fold k=3으로 변경

In [12]:
# 하이퍼파라미터 그리드 (이전 최적값 주변 + SMOTENC 동시 탐색)
param_grid = {
    # RF: 이전 최적 근처를 촘촘히 재탐색
    'clf__n_estimators': [600, 800],
    'clf__max_depth': [16, 24],
    'clf__min_samples_split': [2, 5],
    'clf__min_samples_leaf': [8, 12],        # 합성 노이즈 완충을 위해 약간 키움
    'clf__max_features': ['sqrt'],
    'clf__class_weight': [None],    # 오버샘플링과의 조합 비교
    'clf__max_samples': [None, 0.9],

    # SMOTENC: 데이터 밀도에 민감 → 함께 튜닝 권장
    'smote__k_neighbors': [3, 5],
    'smote__sampling_strategy': ['auto', 0.5],  # 소수 클래스 비율(0.5=40%)
}

# 교차검증 & 그리드서치
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

gcv = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=scoring,
    refit='balanced_acc',     # 최종 선택 기준(두 클래스 균형 성능)
    cv=cv,
    n_jobs=-1,
    verbose=1,
    pre_dispatch='2*n_jobs',  # RAM 초과 억제
    error_score='raise'
)

# 학습
gcv.fit(X_train_raw, y_train)

print("최적 balanced_acc (CV):", round(gcv.best_score_, 4))
print("최적 하이퍼파라미터:", gcv.best_params_)

Fitting 3 folds for each of 128 candidates, totalling 384 fits
최적 balanced_acc (CV): 0.7528
최적 하이퍼파라미터: {'clf__class_weight': None, 'clf__max_depth': 16, 'clf__max_features': 'sqrt', 'clf__max_samples': None, 'clf__min_samples_leaf': 8, 'clf__min_samples_split': 2, 'clf__n_estimators': 600, 'smote__k_neighbors': 3, 'smote__sampling_strategy': 'auto'}


In [13]:
# 평가 (원본 테스트셋: 여기에서는 클래스 불균형이 그대로 남아있음)
best_model = gcv.best_estimator_
y_proba = best_model.predict_proba(X_test_raw)[:, 1]
y_pred  = best_model.predict(X_test_raw)

print("\n=== 테스트 성능 (SMOTENC 포함) ===")
print("Balanced Acc:", round(balanced_accuracy_score(y_test, y_pred), 4))
print("ROC-AUC     :", round(roc_auc_score(y_test, y_proba), 4))
print("클래스 0 Recall:", round(recall_score(y_test, y_pred, pos_label=0), 4))
print("클래스 0 PR-AUC :", round(pr_auc_minority(y_test, y_proba), 4))
print("\n혼동 행렬:\n", confusion_matrix(y_test, y_pred))
print("\n분류 보고서:\n", classification_report(y_test, y_pred, digits=4))


=== 테스트 성능 (SMOTENC 포함) ===
Balanced Acc: 0.7634
ROC-AUC     : 0.84
클래스 0 Recall: 0.7138
클래스 0 PR-AUC : 0.5759

혼동 행렬:
 [[ 429  172]
 [ 465 2022]]

분류 보고서:
               precision    recall  f1-score   support

           0     0.4799    0.7138    0.5739       601
           1     0.9216    0.8130    0.8639      2487

    accuracy                         0.7937      3088
   macro avg     0.7007    0.7634    0.7189      3088
weighted avg     0.8356    0.7937    0.8075      3088



---

# 4. Treshold 튜닝

In [14]:
from sklearn.metrics import precision_recall_curve, fbeta_score, classification_report, confusion_matrix

# 1. 테스트 세트에서 Class 1 확률값 예측
y_proba = gcv.best_estimator_.predict_proba(X_test_raw)[:, 1]  # Class 1 확률

# 2. Precision-Recall Curve 계산
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba)

# 3. F2-score 계산 (Recall 가중치 ↑)
f2_scores = []
for t in thresholds:
    preds = (y_proba >= t).astype(int)
    f2_scores.append(fbeta_score(y_test, preds, beta=2))

f2_scores = np.array(f2_scores)

# 4. 두 가지 기준 찾기
# (1) Recall >= 0.90 중에서 precision 최대인 threshold
target_recall_idx = np.where(recalls >= 0.90)[0]
if len(target_recall_idx) > 0:
    best_idx_recall = target_recall_idx[np.argmax(precisions[target_recall_idx])]
    best_threshold_recall = thresholds[best_idx_recall]
else:
    best_threshold_recall = None

# (2) F2-score 최대 threshold
best_idx_f2 = np.argmax(f2_scores)
best_threshold_f2 = thresholds[best_idx_f2]

print(f"🔹 Best threshold for Recall>=0.90: {best_threshold_recall:.4f}" if best_threshold_recall else "No threshold meets Recall>=0.90")
print(f"🔹 Best threshold for max F2-score: {best_threshold_f2:.4f}")

# 5. 각 threshold에서 성능 비교
for th, name in [(best_threshold_recall, "Recall≥0.90"), (best_threshold_f2, "Max F2")]:
    if th is not None:
        preds = (y_proba >= th).astype(int)
        print(f"\n===== {name} (Threshold={th:.4f}) =====")
        print("Confusion Matrix:")
        print(confusion_matrix(y_test, preds))
        print(classification_report(y_test, preds, digits=4))

🔹 Best threshold for Recall>=0.90: 0.3765
🔹 Best threshold for max F2-score: 0.0903

===== Recall≥0.90 (Threshold=0.3765) =====
Confusion Matrix:
[[ 306  295]
 [ 248 2239]]
              precision    recall  f1-score   support

           0     0.5523    0.5092    0.5299       601
           1     0.8836    0.9003    0.8919      2487

    accuracy                         0.8242      3088
   macro avg     0.7180    0.7047    0.7109      3088
weighted avg     0.8191    0.8242    0.8214      3088


===== Max F2 (Threshold=0.0903) =====
Confusion Matrix:
[[  68  533]
 [  11 2476]]
              precision    recall  f1-score   support

           0     0.8608    0.1131    0.2000       601
           1     0.8229    0.9956    0.9010      2487

    accuracy                         0.8238      3088
   macro avg     0.8418    0.5544    0.5505      3088
weighted avg     0.8302    0.8238    0.7646      3088



# 현재까지 결과:
## 1. 성능 비교

| 모델 | Balanced Acc | ROC-AUC | Class 0 Recall | Class 0 Precision | Class 1 Recall | Class 1 Precision | 특징 |
|------|-------------|---------|----------------|-------------------|----------------|-------------------|------|
| **SMOTENC X** | 0.7606 | **0.8551** | 0.6889 | **0.4982** | 0.8323 | 0.9171 | 기본 모델 |
| **SMOTENC O (모델1)** | 0.7606 | 0.8393 | 0.7121 | 0.4740 | 0.8090 | **0.9208** | 메모리 경고 발생 |
| **SMOTENC O (모델2)** | **0.7634** | 0.8400 | **0.7138** | 0.4799 | 0.8130 | **0.9216** | 튜닝 안정 |
| **Threshold 튜닝 (Recall ≥ 0.90)** | 0.7047 | N/A | 0.5092 | 0.5523 | **0.9003** | 0.8836 | FN 대폭 감소 |
| **Threshold 튜닝 (Max F2)** | **0.5544** | N/A | **0.1131** | **0.8608** | **0.9956** | 0.8229 | 이탈 거의 100% 탐지 |


## 2. 해석

- **목표: 이탈 고객 탐지 (is_churned=1)**  
  - **Recall ≥ 0.90 모델** → Class 1 Recall 90% 달성, FN 크게 줄임. Precision도 0.88로 나쁘지 않음.  
  - **Max F2 모델** → Class 1 Recall 99.6% 달성, 거의 모든 이탈 고객 잡음. 대신 Class 0 Recall이 매우 낮음(잔류 고객 오분류 많음).
- **ROC-AUC & 전반적인 확률 예측력**  
  - SMOTENC 미적용 모델이 가장 높음 (0.8551) → 전체 예측 정확도/순위 기반 평가는 여전히 우위.
- **균형 성능**  
  - Balanced Accuracy는 SMOTENC 모델들이 약간 높지만, threshold 튜닝 모델들은 Class 0 성능이 희생되며 하락.



## 3. 추천
- **이탈 고객을 놓치면 안 되는 경우**  
  - **Max F2 모델**이 최적 → Recall 99.6%, 거의 모든 이탈 포착  
  - 단, 정상 고객을 많이 이탈로 예측하므로, 후속 필터링(비용 고려)이 필요
- **이탈 고객을 많이 잡으면서도 Precision 유지**  
  - **Recall ≥ 0.90 모델**이 현실적 → Recall 90%, Precision 88%
- **균형된 성능 & 안정적인 모델**  
  - SMOTENC 미적용 모델 + Threshold 튜닝이 안전한 선택


## 정리:
- SMOTENC를 적용하는 것이 오히려 약간의 성능 저하가 일어남
    - class_weight='balanced'가 클래스 불균형 문제를 자동으로 보정해 주어서 SMOTENC로 인한 추가 이득이 거의 없는 것으로 추정
    - 동시에 쓰면 효과가 중첩되어 과한 보정이 일어날 수 있고, 그게 성능 하락 원인 중 하나로 보입
- **Recall ≥ 0.90 모델** → 가장 비즈니스적으로 안정적  
- **Max F2 모델** → 캠페인 비용 제한이 없을 때 추천  

---

# 5. SMOTENC 미적용 + RandomForest 튜닝 + 임계값(Threshold) 튜닝

In [17]:
## 전체 과정 재구현

# 1) 라이브러리 & 데이터 로드
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    balanced_accuracy_score,   # 불균형 데이터용: 클래스별 재현율 평균
    roc_auc_score,             # 순위(랭킹) 성능
    average_precision_score,   # PR-AUC 계산(평균정밀도)
    recall_score,              # 재현율
    confusion_matrix,          # 혼동 행렬
    classification_report,     # 정밀도/재현율/F1 종합 리포트
    make_scorer,               # 커스텀 스코어러
    precision_recall_curve,    # 임계값 튜닝용 PR-커브
    fbeta_score,               # F-베타 점수
    precision_score, precision_recall_curve
)

# 데이터 로드
newspaper_df = pd.read_csv('./data/newspaper_preprocessed.csv')

# =========================
# 2) 전처리: 인코딩 스키마
# =========================
# (참고) 'dummy for Children'는 타겟/인코딩 혼선 방지를 위해 제거
if 'dummy for Children' in newspaper_df.columns:
    newspaper_df = newspaper_df.drop('dummy for Children', axis=1)

# 타겟/피처 분리
y = newspaper_df['is_churned'].astype(int)
X = newspaper_df.drop(columns=['is_churned'])

# 범주형: One-Hot / Ordinal 대상 지정 (RandomForest 기준)
onehot_cols = ['Home Ownership', 'County', 'weekly fee', 'Nielsen Prizm']
label_cols  = ['Ethnicity', 'Language', 'City', 'Deliveryperiod', 'Source Channel']

# 수치형 컬럼 자동 탐지
numeric_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()

# 실제 존재하는 컬럼만 사용 (방어코드)
onehot_cols  = [c for c in onehot_cols  if c in X.columns]
label_cols   = [c for c in label_cols   if c in X.columns]
numeric_cols = [c for c in numeric_cols if c in X.columns]

# 인코더 & 스케일러
scaler  = StandardScaler()
ohe     = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1) 

# 열별 전처리기
preprocess_rf = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_cols),  # 수치형 → 스케일링
        ('ohe', ohe, onehot_cols),      # 명목형 → One-Hot
        ('ord', ord_enc, label_cols)    # 고카디널리티 → Ordinal(라벨 인코딩 대체)
    ],
    remainder='drop'
)

# 전체 데이터 인코딩
X_encoded = preprocess_rf.fit_transform(X)
print("Encoded shape:", X_encoded.shape)


# 3) 학습/평가용 분리
# (중요) 불균형 대응을 위해 stratify 사용
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, stratify=y, random_state=42
)
print("Train class ratio (is_churned==1):", y_train.mean().round(4))
print("Test  class ratio (is_churned==1):", y_test.mean().round(4))


# 4) 평가 스코어러 구성
# 클래스 0(PR-AUC) 보조 지표 (필요 시)
def pr_auc_minority_scorer(y_true, y_score_pos1, **kwargs):
    # y_score_pos1: 클래스 1 확률 → 클래스 0 기준으로 보려면 반전
    return average_precision_score(1 - y_true, 1 - y_score_pos1)

scoring = {
    'balanced_acc': make_scorer(balanced_accuracy_score),      # 모델 선택 기준
    'roc_auc': 'roc_auc',                                      # 전체 랭킹 성능
    'recall_minority': make_scorer(recall_score, pos_label=0), # 클래스 0 재현율
    'pr_auc_minority': make_scorer(pr_auc_minority_scorer),    # 클래스 0 PR-AUC
}


# 5) RandomForest + GridSearchCV
# (안정성) 중첩 병렬 과도 방지를 위해 RF는 n_jobs=1, GridSearchCV에서 병렬화 (-1)
rf = RandomForestClassifier(random_state=42, n_jobs=1, bootstrap=True)
pipe = Pipeline([('clf', rf)])

param_grid = {
    'clf__n_estimators': [400, 800],
    'clf__max_depth': [None, 16, 24],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4, 8],
    'clf__max_features': ['sqrt', 'log2', 0.5],
    'clf__class_weight': ['balanced'],
    'clf__max_samples': [None, 0.7, 0.9],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
gcv = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=scoring,
    refit='balanced_acc',  # 최종 선택 기준
    cv=cv,
    n_jobs=-1,
    verbose=1,
    pre_dispatch='2*n_jobs'
)

gcv.fit(X_train, y_train)

print("\n[GridSearchCV 결과]")
print("최적 balanced_acc (CV):", round(gcv.best_score_, 4))
print("최적 하이퍼파라미터:", gcv.best_params_)

# 6) 기본 임계값(0.5) 테스트 성능
best_model = gcv.best_estimator_
y_proba_test = best_model.predict_proba(X_test)[:, 1]
y_pred_test  = best_model.predict(X_test)

print("\n=== [테스트] 기본 임계값 0.5 ===")
print("Balanced Acc:", round(balanced_accuracy_score(y_test, y_pred_test), 4))
print("ROC-AUC     :", round(roc_auc_score(y_test, y_proba_test), 4))
print("Class 0 Recall:", round(recall_score(y_test, y_pred_test, pos_label=0), 4))
print("Class 0 PR-AUC:", round(pr_auc_minority_scorer(y_test, y_proba_test), 4))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("\nClassification Report:\n", classification_report(y_test, y_pred_test, digits=4))


# 7) 임계값 튜닝 (양성=클래스 1: 이탈 고객)
# 7-1) 훈련 세트 내부에서 검증 세트 분리 (누수 방지용 권장 절차)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

# (선택) 엄밀히 하려면: 최적 하이퍼파라미터로 새 모델 생성 후 X_tr로 재학습
# 여기서는 이미 gcv.best_estimator_가 X_train 전체로 학습되어 있으므로 그대로 사용
# best_model.fit(X_tr, y_tr)  # 엄밀 재학습을 원하면 주석 해제

# 7-2) 검증 세트에서 PR-커브로 임계값 후보 탐색
val_proba_cls1 = best_model.predict_proba(X_val)[:, 1]
prec_arr, rec_arr, thr_arr = precision_recall_curve(y_val, val_proba_cls1)

# (A) 목표 Recall(예: 0.90) 이상 중 Precision 최대 임계값
target_recall = 0.90
candidates_idx = np.where(rec_arr[:-1] >= target_recall)[0]
if len(candidates_idx) > 0:
    best_idx_recall = candidates_idx[np.argmax(prec_arr[candidates_idx])]
    thr_recall = thr_arr[best_idx_recall]
else:
    thr_recall = 0.5  # 후보가 없으면 기본값 유지

# (B) F2 점수(Recall 가중↑) 최대 임계값
f2_scores = []
for t in thr_arr:
    preds_val = (val_proba_cls1 >= t).astype(int)
    f2_scores.append(fbeta_score(y_val, preds_val, beta=2))
f2_scores = np.array(f2_scores)
best_idx_f2 = np.argmax(f2_scores[:-1])  # 마지막 포인트 제외
thr_f2 = thr_arr[best_idx_f2]

print(f"\n[VAL] 목표 Recall≥{target_recall:.2f} 임계값: {thr_recall:.4f}")
print(f"[VAL] F2 최대 임계값: {thr_f2:.4f}")


# 8) 테스트 세트에서 임계값 적용 평가
def eval_with_threshold(y_true, proba_cls1, thr, tag=""):
    y_pred_thr = (proba_cls1 >= thr).astype(int)
    print(f"\n=== [테스트] {tag} (Threshold={thr:.4f}) ===")
    print("Class 1 Precision:", round(precision_score(y_true, y_pred_thr, pos_label=1), 4))
    print("Class 1 Recall   :", round(recall_score(y_true, y_pred_thr, pos_label=1), 4))
    print("Balanced Acc     :", round(balanced_accuracy_score(y_true, y_pred_thr), 4))
    print("ROC-AUC (unchanged):", round(roc_auc_score(y_true, proba_cls1), 4))  # ROC-AUC는 임계값 무관
    print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred_thr))
    print("\nClassification Report:\n", classification_report(y_true, y_pred_thr, digits=4))

proba_test_cls1 = best_model.predict_proba(X_test)[:, 1]
eval_with_threshold(y_test, proba_test_cls1, thr_recall, tag=f"Recall≥{target_recall:.2f}")
eval_with_threshold(y_test, proba_test_cls1, thr_f2,     tag="Max F2")

Encoded shape: (15438, 38)
Train class ratio (is_churned==1): 0.8054
Test  class ratio (is_churned==1): 0.8054
Fitting 5 folds for each of 648 candidates, totalling 3240 fits

[GridSearchCV 결과]
최적 balanced_acc (CV): 0.7672
최적 하이퍼파라미터: {'clf__class_weight': 'balanced', 'clf__max_depth': 24, 'clf__max_features': 'sqrt', 'clf__max_samples': None, 'clf__min_samples_leaf': 8, 'clf__min_samples_split': 2, 'clf__n_estimators': 400}

=== [테스트] 기본 임계값 0.5 ===
Balanced Acc: 0.7606
ROC-AUC     : 0.8551
Class 0 Recall: 0.6889
Class 0 PR-AUC: 0.6023

Confusion Matrix:
 [[ 414  187]
 [ 417 2070]]

Classification Report:
               precision    recall  f1-score   support

           0     0.4982    0.6889    0.5782       601
           1     0.9171    0.8323    0.8727      2487

    accuracy                         0.8044      3088
   macro avg     0.7077    0.7606    0.7254      3088
weighted avg     0.8356    0.8044    0.8154      3088


[VAL] 목표 Recall≥0.90 임계값: 0.4753
[VAL] F2 최대 임계값: 0.2652


---

| 모델 | SMOTENC | Balanced Acc | ROC-AUC | Class 0 Recall | Class 0 Precision | Class 1 Recall | Class 1 Precision | 특징 |
|------|---------|-------------|---------|----------------|-------------------|----------------|-------------------|------|
| **기본 모델** | X | 0.7606 | **0.8551** | 0.6889 | **0.4982** | 0.8323 | 0.9171 | 기본 세팅, 두 클래스 균형 양호 |
| **SMOTENC 적용 (모델1)** | O | 0.7606 | 0.8393 | 0.7121 | 0.4740 | 0.8090 | **0.9208** | 메모리 경고 발생 |
| **SMOTENC 적용 (모델2)** | O | **0.7634** | 0.8400 | **0.7138** | 0.4799 | 0.8130 | **0.9216** | SMOTENC + 안정 튜닝 |
| **Threshold 튜닝 (Recall ≥ 0.90)** | X | 0.7554 | **0.8551** | 0.6572 | 0.5204 | **0.8536** | 0.9116 | FN 감소, Precision 유지 |
| **Threshold 튜닝 (Max F2)** | X | **0.6292** | **0.8551** | **0.2845** | **0.7246** | **0.9739** | 0.8492 | 이탈 탐지 극대화, 정상 고객 오탐 증가 |
| **Threshold 튜닝 (Recall ≥ 0.90)** | O | 0.7047 | N/A | 0.5092 | 0.5523 | **0.9003** | 0.8836 | FN 대폭 감소, Precision 소폭 하락 |
| **Threshold 튜닝 (Max F2)** | O | **0.5544** | N/A | **0.1131** | **0.8608** | **0.9956** | 0.8229 | 거의 모든 이탈 탐지, 정상 고객 오탐 매우 많음 |

## 📊 모델 비교 및 추천

### 1. 해석 기준
- **프로젝트 목표**: 이탈 고객(`is_churned == 1`)을 최대한 놓치지 않는 것 (Recall↑),  
  그러나 **정상 고객을 너무 많이 오탐**하면 비용 증가 → Precision도 고려 필요.
- **Balanced Accuracy**: 두 클래스 재현율 평균. 데이터 불균형 시 유용.
- **ROC-AUC**: 분류 전체 성능.  
- **Threshold 튜닝**: Recall과 Precision의 균형을 의도적으로 조정.

### 2. 핵심 비교
| 후보 | 장점 | 단점 | 종합 평가 |
|------|------|------|-----------|
| **기본 모델 (SMOTENC X)** | 높은 ROC-AUC (0.8551), 균형 잡힌 성능, Class 1 Recall 0.8323 | Class 0 Recall 0.6889 → 정상 고객 탐지 조금 부족 | 안정적이고 해석 쉬움 |
| **SMOTENC 적용 (모델2)** | Balanced Acc 최고 (0.7634), Class 0 Recall 0.7138 | ROC-AUC가 기본 모델보다 약간 낮음 | 정상 고객 탐지 조금 개선 |
| **Threshold 튜닝 (Recall≥0.90, X)** | Class 1 Recall 0.8536로 목표 충족, Precision 0.9116 유지 | Balanced Acc 소폭 감소 | **목표(이탈 방지)와 Precision 균형**이 가장 적합 |
| **Threshold 튜닝 (Max F2, X)** | Class 1 Recall 0.9739 (거의 모든 이탈 탐지) | Class 0 Recall 0.2845로 정상 고객 오탐 심각 | 운영 환경에서는 비효율적 |
| **Threshold 튜닝 (Recall≥0.90, O)** | Class 1 Recall 0.9003 | Class 0 Recall 0.5092로 매우 낮음 | 이탈 탐지는 잘하나 정상 고객 오탐 심각 |
| **Threshold 튜닝 (Max F2, O)** | Class 1 Recall 0.9956 (거의 100%) | Class 0 Recall 0.1131로 사실상 정상 고객 필터링 불가능 | 운영 불가능 수준 |

### 3. 추천
- **운영 환경 추천**:  
  **기본 모델(SMOTENC X) + Threshold 튜닝(Recall ≥ 0.90)**  
  → 이탈 고객 Recall 0.8536, Precision 0.9116로 균형 우수.
- **분석/실험 목적**:  
  SMOTENC 적용 모델2 (Balanced Acc 최고)도 보조 지표로 사용 가능.

✅ 결론:  
**SMOTENC 미적용 + Recall ≥ 0.90 Threshold 튜닝**이  
운영상 Precision과 Recall 균형에서 가장 효율적.